In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples

In [53]:
df = pd.read_csv("coffee_analysis.csv")
columns = ["name", "roast", "loc_country", "100g_USD", "rating", "desc_1", "desc_2", "desc_3"]
dffiltered = df[columns]
dffiltered

,name,roast,loc_country,100g_USD,rating,desc_1,desc_2,desc_3
0,“Sweety” Espresso Blend,Medium-Light,Hong Kong,14.32,95,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,Flora Blend Espresso,Medium-Light,Hong Kong,9.05,94,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,Ethiopia Shakiso Mormora,Medium-Light,United States,4.70,92,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,Ethiopia Suke Quto,Medium-Light,United States,4.19,92,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,Ethiopia Gedeb Halo Beriti,Medium,United States,4.85,94,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...
...,...,...,...,...,...,...,...,...
2090,Finca Patzibir,Medium-Light,Guatemala,4.70,92,"Crisply sweet, nut-toned. Almond brittle, pie ...",Produced by Andres Fahsen of Finca Patzibar en...,"A quietly confident, sweetly nut-toned Guatema..."
2091,Proyecto Aurora,Medium-Light,Guatemala,2.94,93,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",Produced at Beneficio La Esperanza from trees ...,"A deeply floral, richly chocolaty Guatemala cu..."
2092,Finca El Potrero,Medium-Light,Guatemala,2.94,93,"High-toned, enticingly sweet. Black cherry, na...",Produced by Josué Morales of Finca El Potrero...,"A bright, balanced, juicy Guatemala cup driven..."
2093,Chacayá Santiago Atitlán,Medium-Light,Guatemala,4.70,93,"Vibrantly sweet, subtly nuanced. Apricot, dar...",Produced by smallholding farmers of Asociación...,"Balanced, bright, invigoratingly crisp, with t..."


In [54]:
nltk.download("stopwords")
nltk.download("wordnet")
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\infin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\infin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# REMOVING THE FLUFF (REGEX AND OTHER STUFF)

In [56]:
import re
import string
import numpy as np
df_1 = dffiltered.copy()

#converts digits and other fluff into string format so regex can do its work
df_1["desc_1"]=df_1["desc_1"].apply(str)
df_1["desc_2"]=df_1["desc_2"].apply(str)
df_1["desc_3"]=df_1["desc_3"].apply(str)

def processed_tweets(review):
    temp = review
    temp = re.sub("\d+", "", temp)
    temp = re.sub(r'@\w+', '', temp)
    temp = re.sub(r'[^\w\s]','',temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = " ".join(word for word in temp.split() if word not in stop_words)
    return temp

#apply the regex to process the fluff in the descriptions
df_1["processeddesc_1"] = df_1["desc_1"].apply(lambda x: processed_tweets(x))
df_1["processeddesc_2"] = df_1["desc_2"].apply(lambda x: processed_tweets(x))
df_1["processeddesc_3"] = df_1["desc_3"].apply(lambda x: processed_tweets(x))
df_1["Full Desc"] = df_1["processeddesc_1"] + "." + df_1["processeddesc_2"] + "." + df_1["processeddesc_3"]


for item in ["name", "roast", "loc_country","100g_USD", "rating"]:
    df_1[item].replace('', np.nan, inplace=True)
    df_1.dropna(subset=[item], inplace=True)
    


#create another dataframe df_2 so that only the full description is selected
columns = ["name", "roast", "loc_country", "100g_USD", "rating", "Full Desc"]
df_2 = df_1[columns]
df_2

,name,roast,loc_country,100g_USD,rating,Full Desc
0,“Sweety” Espresso Blend,Medium-Light,Hong Kong,14.32,95,Evaluated espresso Sweettoned deeply rich choc...
1,Flora Blend Espresso,Medium-Light,Hong Kong,9.05,94,Evaluated espresso Sweetly tart floraltoned Ho...
2,Ethiopia Shakiso Mormora,Medium-Light,United States,4.70,92,Crisply sweet cocoatoned Lemon blossom roasted...
3,Ethiopia Suke Quto,Medium-Light,United States,4.19,92,Delicate sweetly spicetoned Pink peppercorn da...
4,Ethiopia Gedeb Halo Beriti,Medium,United States,4.85,94,Deeply sweet subtly pungent Honey pear tangeri...
...,...,...,...,...,...,...
2090,Finca Patzibir,Medium-Light,Guatemala,4.70,92,Crisply sweet nuttoned Almond brittle pie cher...
2091,Proyecto Aurora,Medium-Light,Guatemala,2.94,93,Chocolaty floralframed Cocoa nib honeysuckle d...
2092,Finca El Potrero,Medium-Light,Guatemala,2.94,93,Hightoned enticingly sweet Black cherry narcis...
2093,Chacayá Santiago Atitlán,Medium-Light,Guatemala,4.70,93,Vibrantly sweet subtly nuanced Apricot dark ch...


In [51]:
IDEAS: relation between type of roast with respect to sentiments
    relation between the perceived ratings of customers and the price of the beans per 100g
    
    